In [1]:
import numpy as np
from mydx import mydx, fmm_pcm
#mydx.init()
#nbasis, ngrid, nsph = mydx.get_sizes()
#lmax = int(nbasis**0.5)-1
#print(nbasis, lmax, ngrid)

In [2]:
# COMPLEX
lmax = 2
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([.0, 0, 0.5])
sph2_r = 1
src1 = np.array([0.0, 0.5, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_complex(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_complex(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_complex_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_complex_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_complex_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)

0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214
0.3127313958110473 0.31234752377721214
0.3126262395925993 0.31234752377721214


In [3]:
# REAL
lmax = 17
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1
sph2_c = np.array([1.0, 1, 1])
sph2_r = 1
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2m_real(src1-sph1_c, lmax)
coef2_m = fmm_pcm.p2m_real(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real(dst1-sph1_c, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real(dst1-sph2_c, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)
coef1_m_normal = fmm_pcm.p2m_real_normal(src1-sph1_c, lmax)
coef2_m_normal = fmm_pcm.p2m_real_normal(src1-sph2_c, lmax)
print(fmm_pcm.m2p_real_normal(dst1-sph1_c, lmax, coef1_m_normal), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.m2p_real_normal(dst1-sph2_c, lmax, coef2_m_normal), np.linalg.norm(src1-dst1)**-1)
#coef2_m_m2m = fmm_pcm.m2m_complex_translate_z(sph1_c[2]-sph2_c[2], lmax, coef1_m)
#print(coef1_m)
#print(coef2_m)
#print(coef2_m_m2m)
#print(fmm_pcm.m2p_complex_normal(dst1-sph2_c, lmax, coef2_m_m2m), np.linalg.norm(src1-dst1)**-1)

0.3254722774521031 0.32547227745205964
0.32525817347924185 0.32547227745205964
0.3254722774521031 0.32547227745205964
0.3252581734792418 0.32547227745205964


In [4]:
# Normalized real spherical harmonics
lmax = 60
sph1_c = np.array([0.0, 0, 0])
sph1_r = 1.5
sph2_c = np.array([1.0, 1, 1])
sph2_r = 2*3**0.5 + 1.5
src1 = np.array([0.0, 0.8, 0.0])
dst1 = np.array([2., 2., 2.])
coef1_m = fmm_pcm.p2o(src1-sph1_c, sph1_r, lmax)
coef2_m = fmm_pcm.p2o(src1-sph2_c, sph2_r, lmax)
print(fmm_pcm.o2p(dst1-sph1_c, sph1_r, lmax, coef1_m), np.linalg.norm(src1-dst1)**-1)
print(fmm_pcm.o2p(dst1-sph2_c, sph2_r, lmax, coef2_m), np.linalg.norm(src1-dst1)**-1)

0.3254722774520597 0.32547227745205964
0.3254718954621216 0.32547227745205964


In [7]:
# Compute M2M matrix by using P2M and M2P
lmax = 20
nparticles = 4000
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 20
sph2_c = np.array([0, 20, 0], dtype=np.float64)
sph2_r = 40
particles = np.zeros((nparticles, 3), dtype=np.float64)
mat1 = np.zeros((nparticles, (lmax+1)**2), dtype=np.complex128)
mat2 = np.zeros((nparticles, (lmax+1)**2), dtype=np.complex128)
np.random.seed(0)
for i in range(nparticles):
    tmp = np.random.randn(3)
    tmp = tmp / np.linalg.norm(tmp)
    particles[i] = sph1_c + 0.5 * sph1_r * tmp
    mat1[i] = fmm_pcm.p2m_complex(particles[i]-sph1_c, lmax)
    mat2[i] = fmm_pcm.p2m_complex(particles[i]-sph2_c, lmax)
q, r = np.linalg.qr(mat1)
#print(r)
m2m = np.linalg.solve(r, q.T @ mat2).T
print(np.linalg.norm(m2m @ mat1[0] - mat2[0]) / np.linalg.norm(mat2[0]))
#print(m2m)
print(np.max(m2m[1:]), np.min(m2m))
mat3 = fmm_pcm.m2m_baseline(sph1_c-sph2_c, lmax, mat1[0])
#print(m2m[10,0])
print(mat1[0][:10])
print(mat2[0][:10])
print(mat3[:10])
print(np.linalg.norm(mat3-mat2[0]) / np.linalg.norm(mat2[0]))

35.82311917903217
(9.152107628309846e+25+1.90714770298832e+25j) (-3.964049478124946e+26+4.946005865703125e+25j)
[  1.          +0.j           6.06498385  +1.37577947j
   4.75882376  +0.j           6.06498385  -1.37577947j
  42.73289161 +20.43873904j  49.99077818 +11.33989606j
 -16.0303946   +0.j          49.99077818 -11.33989606j
  42.73289161 -20.43873904j 298.29085106+235.93137483j]
[ 1.00000000e+00   +0.j          6.06498385e+00  -12.76635615j
  4.75882376e+00   +0.j          6.06498385e+00  +12.76635615j
 -1.54557684e+02 -189.65846432j  4.99907782e+01 -105.22700387j
 -1.77117475e+02   +0.j          4.99907782e+01 +105.22700387j
 -1.54557684e+02 +189.65846432j -4.33598218e+03+1062.3114551j ]
[ 1.00000000e+00   +0.j          6.06498385e+00  -12.76635615j
  4.75882376e+00   +0.j          6.06498385e+00  +12.76635615j
 -1.54557684e+02 -189.65846432j  4.99907782e+01 -105.22700387j
 -1.77117475e+02   +0.j          4.99907782e+01 +105.22700387j
 -1.54557684e+02 +189.65846432j -4.33598218e

In [14]:
# Compute M2M matrix by using P2M and M2P
lmax = 2
nparticles = 40
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 2
sph2_c = np.array([0, 2, 0], dtype=np.float64)
sph2_r = 4
particles = np.zeros((nparticles, 3), dtype=np.float64)
mat1 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
mat2 = np.zeros((nparticles, (lmax+1)**2), dtype=np.float64)
np.random.seed(0)
for i in range(nparticles):
    tmp = np.random.randn(3)
    tmp = tmp / np.linalg.norm(tmp)
    particles[i] = sph1_c + 0.5 * sph1_r * tmp
    mat1[i] = fmm_pcm.p2o(particles[i]-sph1_c, sph1_r, lmax)
    mat2[i] = fmm_pcm.p2o(particles[i]-sph2_c, sph2_r, lmax)
q, r = np.linalg.qr(mat1)
#print(r)
m2m = np.linalg.solve(r, q.T @ mat2).T
print(np.linalg.norm(m2m @ mat1[0] - mat2[0]) / np.linalg.norm(mat2[0]))
#print(m2m)
print(np.max(m2m[1:]), np.min(m2m))
mat3 = fmm_pcm.o2o_baseline(sph1_c-sph2_c, sph1_r, sph2_r, lmax, mat1[0])
#print(m2m[10,0])
print(mat1[0][:10])
print(mat2[0][:10])
print(mat3[:10])
print(np.linalg.norm(mat3-mat2[0]) / np.linalg.norm(mat2[0]))

5.632702144529819e-16
0.8660254037844388 -0.5590169943749475
[ 0.28209479 -0.04753238  0.11625866  0.20954164 -0.04558156 -0.02528973
 -0.02527926  0.11148719  0.09530098]
[ 0.28209479  0.22053507  0.05812933  0.10477082  0.10574195  0.05866814
 -0.0698267   0.0278718  -0.0861719 ]
[ 0.28209479 -0.02376619  0.05812933  0.2775179  -0.68044193 -0.00632243
 -0.07431997  0.05440409  0.05763988]
2.0751669398500905


In [7]:
x = -4.7532375840547543E-002*1.4142135623730951*0.70710678118654746/2*2.0466534158929770*0.48860251190291992
print(x)

-0.02376618792027377


In [12]:
# CHECK O2O (scaled M2M)
lmax = 10
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 2
sph2_c = np.array([0, 2, 0], dtype=np.float64)
sph2_r = 4
dst1 = np.array([4.2, 0., 2.])
coef1_m = np.random.randn((lmax+1)**2)
coef2_m = fmm_pcm.o2o_baseline(sph1_c-sph2_c, sph1_r, sph2_r, lmax, coef1_m)
print(fmm_pcm.o2p(dst1-sph1_c, sph1_r, lmax, coef1_m), fmm_pcm.o2p(dst1-sph2_c, sph2_r, lmax, coef2_m))

0.5330690986592658 15312500861798.676


In [10]:
# Check O2O matrix
lmax = 18
sph1_c = np.array([0, 0, 0], dtype=np.float64)
sph1_r = 0.2
sph2_c = np.array([0, 0, 0.2], dtype=np.float64)
sph2_r = 0.4
sph3_c = np.array([0, 0, 0], dtype=np.float64)
sph3_r = 20000
sph4_c = np.array([0, 0, 20000], dtype=np.float64)
sph4_r = 40000
o2o_mat1 = fmm_pcm.o2o_matrix(sph1_c-sph2_c, sph1_r, sph2_r, lmax)
o2o_mat2 = fmm_pcm.o2o_matrix(sph3_c-sph4_c, sph3_r, sph4_r, lmax)
print(np.linalg.norm(o2o_mat1-o2o_mat2), np.linalg.norm(o2o_mat1))
#print(np.linalg.svd(o2o_mat)[1])

0.0 5.8696047859167075
